In [7]:
from pathlib import Path
from FISHClass.datasets import MYCN
from torch.utils.data import DataLoader
import yaml
from baseline import models
from FISHClass.utils.evaluation import get_top_model
from FISHClass.evaluation.evaluate_test_set import predict_test, predict_test_baseline
from FISHClass.models import CombinedModel
from FISHClass.utils.device import best_gpu
import os
import torch
import FISHClass

In [8]:
DATASET = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/MYCN_SpikeIn/TRAINING.h5"
OUT = "/home/simon_g/src/eval/test_set"

with open("/home/simon_g/src/FISHClass/evaluation/model_evaluation.yaml") as f:

    yaml_data = yaml.load(f, Loader=yaml.FullLoader)["trained_model_paths"]

for model_name, model_items in yaml_data.items():
    
    if not isinstance(model_items, str):
        
        if model_items["model_type"] == "AreaModel":
            model = getattr(models, model_items["model_type"])(**model_items["AreaModel_kwargs"])
            results = predict_test_baseline(model, DATASET, dataset_kwargs={"norm_type": None, "transform": None}, save2h5=True, save_path=os.path.join(OUT, f"{model_name}_results.h5"))
    
    
        elif model_items["model_type"] == "SpotdetectionModel":

            model = getattr(models, model_items["model_type"])(**model_items["SpotdetectionModel_kwargs"])
            results = predict_test_baseline(model, DATASET, dataset_kwargs={"norm_type": None, "transform": None}, save2h5=True, save_path=os.path.join(OUT, f"{model_name}_results.h5"))
        
    else:
        try:
            model = torch.load(get_top_model(model_items))["model"]
        except:
            model = torch.load(get_top_model(model_items))
            
        print(type(model))
        results= predict_test(model, DATASET, device=best_gpu(), batch_size=16, 
                              dataset_kwargs={"double_return": isinstance(model, (FISHClass.ModelZoo.FeaturespaceClassifier.FeaturespaceClassifier, FISHClass.ModelZoo.WeightedFeaturespaceClassifier.WeightedFeaturespaceClassifier)), 
                                              "norm_type": model.norm_type, 
                                              "mask": model.mask, 
                                              "channels": model.channels, 
                                              "transform": None}, 
                              save2h5=True, 
                              save_path=os.path.join(OUT, f"{model_name}_results.h5"))

<class 'FISHClass.ModelZoo.CombinedModel.CombinedModel'>
Using cuda:3 for calculation


100%|██████████| 61/61 [00:31<00:00,  1.95it/s]


<class 'FISHClass.ModelZoo.CombinedModel.CombinedModel'>
Using cuda:1 for calculation


100%|██████████| 61/61 [00:37<00:00,  1.62it/s]


<class 'FISHClass.ModelZoo.ClassificationCNN.ClassificationCNN'>
couldtn redefine
Using cuda:3 for calculation


...

100%|██████████| 61/61 [00:07<00:00,  7.92it/s]


<class 'FISHClass.ModelZoo.FeaturespaceClassifier.FeaturespaceClassifier'>
Using cuda:3 for calculation


100%|██████████| 61/61 [00:31<00:00,  1.94it/s]


<class 'FISHClass.ModelZoo.FeaturespaceClassifier.FeaturespaceClassifier'>
Using cuda:2 for calculation


100%|██████████| 31/31 [00:08<00:00,  3.47it/s]


In [11]:
import h5py
import numpy as np

files = list(Path("/home/simon_g/src/eval/test_set").glob("*.h5"))
print(files)

[PosixPath('/home/simon_g/src/eval/test_set/Basic-Classifier_results.h5'), PosixPath('/home/simon_g/src/eval/test_set/CNN-RGB_results.h5'), PosixPath('/home/simon_g/src/eval/test_set/CNN-GREEN-MASK_results.h5'), PosixPath('/home/simon_g/src/eval/test_set/SpotdetectionModel_results.h5'), PosixPath('/home/simon_g/src/eval/test_set/AreaModel_results.h5'), PosixPath('/home/simon_g/src/eval/test_set/LSTM-Classifier_results.h5'), PosixPath('/home/simon_g/src/eval/test_set/FS-GREEN-MASK_results.h5'), PosixPath('/home/simon_g/src/eval/test_set/FS-RGB_results.h5')]


In [13]:
import pandas as pd

results_dict = {}
for file in files: 
    
    with h5py.File(file) as fout:
        
        target = np.array(fout["TARGET"])
        pred = np.array(fout["PRED"])
        
    TP = len(np.where(np.logical_and(target == 1, pred == 1))[0])
    TN = len(np.where(np.logical_and(target == 0, pred == 0))[0])
    FP = len(np.where(np.logical_and(target == 0, pred == 1))[0])
    FN = len(np.where(np.logical_and(target == 1, pred == 0))[0])
    SPEC = np.round(TN / (TN + FP)*100,2)
    
    # Precision = TP / (TP + FP) 
    # Recall = TP / (TP + FN) 
    # F1 Score = 2 * Precision * Recall / (Precision + Recall)
    
    precision = np.round(TP/(TP+FP)*100,2)
    recall = np.round(TP/(TP+FN)*100,2)
    F1 = np.round(2*precision*recall/(precision+recall),2)
    
    print(f"{file.stem:<30}", " PRECISION: ", f"{precision:<8}", "RECALL/SENSITIVITY: ", f"{recall:<8}", "F1: ", f"{F1:<8}", "SPECIFICITY: ", f"{SPEC:<8}")
    results_dict[f"{file.stem.replace('_results','')}"] = {
        "precision": precision,
        "recall": recall,
        "F1": F1,
        "specificity": SPEC,
        "sensitivity": recall
    }
    
results_df = pd.DataFrame(results_dict)
results_df.to_excel("/home/simon_g/src/results/EVALUATION/metric_results.xlsx", sheet_name="metrics")

Basic-Classifier_results        PRECISION:  85.15    RECALL/SENSITIVITY:  88.25    F1:  86.67    SPECIFICITY:  85.4    
CNN-RGB_results                 PRECISION:  98.92    RECALL/SENSITIVITY:  98.29    F1:  98.6     SPECIFICITY:  98.99   
CNN-GREEN-MASK_results          PRECISION:  99.14    RECALL/SENSITIVITY:  98.72    F1:  98.93    SPECIFICITY:  99.19   
SpotdetectionModel_results      PRECISION:  83.51    RECALL/SENSITIVITY:  51.92    F1:  64.03    SPECIFICITY:  90.26   
AreaModel_results               PRECISION:  91.19    RECALL/SENSITIVITY:  88.46    F1:  89.8     SPECIFICITY:  91.89   
LSTM-Classifier_results         PRECISION:  97.57    RECALL/SENSITIVITY:  94.23    F1:  95.87    SPECIFICITY:  97.77   
FS-GREEN-MASK_results           PRECISION:  99.35    RECALL/SENSITIVITY:  98.29    F1:  98.82    SPECIFICITY:  99.39   
FS-RGB_results                  PRECISION:  98.49    RECALL/SENSITIVITY:  97.86    F1:  98.17    SPECIFICITY:  98.58   


In [ ]:
import h5py
import numpy as np

files = list(Path("/home/simon_g/src/eval/test_set").glob("*.h5"))
print(files)

In [ ]:

import pandas as pd

results_dict = {}
for file in files: 
    
    with h5py.File(file) as fout:
        
        target = np.array(fout["TARGET"])
        pred = np.array(fout["PRED"])
        
    TP = len(np.where(np.logical_and(target == 1, pred == 1))[0])
    TN = len(np.where(np.logical_and(target == 0, pred == 0))[0])
    FP = len(np.where(np.logical_and(target == 0, pred == 1))[0])
    FN = len(np.where(np.logical_and(target == 1, pred == 0))[0])
    SPEC = np.round(TN / (TN + FP)*100,2)
    
    # Precision = TP / (TP + FP) 
    # Recall = TP / (TP + FN) 
    # F1 Score = 2 * Precision * Recall / (Precision + Recall)
    
    precision = np.round(TP/(TP+FP)*100,2)
    recall = np.round(TP/(TP+FN)*100,2)
    F1 = np.round(2*precision*recall/(precision+recall),2)
    
    print(f"{file.stem:<30}", " PRECISION: ", f"{precision:<8}", "RECALL/SENSITIVITY: ", f"{recall:<8}", "F1: ", f"{F1:<8}", "SPECIFICITY: ", f"{SPEC:<8}")
    results_dict[f"{file.stem.replace('_results','')}"] = {
        "precision": precision,
        "recall": recall,
        "F1": F1,
        "specificity": SPEC,
        "sensitivity": recall
    }
    
results_df = pd.DataFrame(results_dict)
results_df.to_excel("/home/simon_g/src/results/EVALUATION/metric_results.xlsx", sheet_name="metrics")

# Creation of all plots

In [55]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

df_pred = pd.read_excel("/home/simon_g/src/results/EVALUATION/results_paper.xlsx", sheet_name="predictions", index_col=0).T
df_diff = pd.read_excel("/home/simon_g/src/results/EVALUATION/results_paper.xlsx", sheet_name="differences", index_col=0).T

In [77]:
df_pred = df_pred.reindex(
    ['AreaModel', 'SpotdetectionModel', 'BasicClassifier', 'LSTMClassifier', 'ClassificationCNN', 'FeaturespaceClassifier', 'WeightedFeaturespaceClassifier']
    )

In [78]:
df_diff = df_diff.reindex(
    ['AreaModel', 'SpotdetectionModel', 'BasicClassifier', 'LSTMClassifier', 'ClassificationCNN', 'FeaturespaceClassifier', 'WeightedFeaturespaceClassifier']
    )